In [ ]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [1]:
import json
import os
import re

from ollama import chat
from ollama import ChatResponse

In [27]:
MODEL_NAME = 'qwen3:14b' # qwen3:4b, qwen3:8b, qwen3:14b

# Define the API clients

In [3]:
from dotenv import load_dotenv
import os

load_dotenv(os.path.expanduser("~/Desktop/dot-env-api-keys/my-api-keys.env"))

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
# Tavily web search

from tavily import TavilyClient

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# What is the objective?

- Get an email from a customer
- Categorize the email
- Decide if research is needed before replying
- Do research if needed
- Write a draft reply

# Create a list of agents

AGENTS
1. email_cat_agent
2. router_agent (route to research_agent or email_writer_agent)
3. research_agent
4. email_writer_agent

TOOLS
- Tavily search (used by research_agent)


# Helper functions

In [5]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [9]:
def initialize_message_history(system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history

In [7]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [8]:
def replace_items_in_string(input_string):

    """
    Sometimes models produce json style output strings that cannot be parsed.
    This functon extracts the "value" from the json string.
    Extracts the value from a json string.
    Works only if the json string has one key/value pair.
    """

    # The items in this list get replaced with nothing.
    # The key needs to be added to this list.
    items_to_replace = ["```", "json", "{", "}", '"email_draft": "',
                        '"translation": "', '"#"', '"category": "', '"router_decision": "', "\\r"]

    modified_string = input_string
    for item in items_to_replace:
        regex = re.compile(re.escape(item))  # Create a regex pattern for each item
        modified_string = regex.sub("", modified_string)  # Replace item with an empty string

    modified_string = modified_string.strip()

    # Slice to get the string from the start up to the second last character
    if len(modified_string) > 1:
        modified_string = modified_string[:-1]


    return modified_string



# Example usage
input_string = '{"email_draft": "some-value"}'
print(replace_items_in_string(input_string))

some-value


# Set up the LLM

In [11]:
# Function to separate the thinking and the response

def process_response(text):
    
    text1 = text.split('</think>')[0]
    text2 = text.split('</think>')[1]
    
    thinking_text = text1 + '</think>'
    response_text = text2.strip()

    return thinking_text, response_text

In [24]:
def make_llm_api_call(message_history):

    model_name = MODEL_NAME

    response: ChatResponse = chat(model=model_name, 
                                  messages=message_history,
                                )

    output_text = response['message']['content']

    thinking_text, response_text = process_response(output_text)

    print(thinking_text)

    return response_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response_text = make_llm_api_call(message_history)

print(response_text)

#print(response['message']['content'])

<think>
Okay, the user asked, "What's your name?" I need to respond as Molly. Let me make sure I'm clear and friendly. I should start by stating my name directly. Maybe add a bit of personality to make the interaction more engaging. I can mention that I'm here to help with any questions they might have. Keep it simple and straightforward. No need for any extra information unless they ask. Just a polite and helpful reply.
</think>
Hi there! My name is Molly. I'm here to help with any questions you might have. How can I assist you today? 😊


# Set up the tools



In [13]:
def run_tavily_search(query, num_results=5):

    """
    Uses the Tavily API to run a web search
    Args:
        query (str): The user query
        num_results (int): Num search results
    Returns:
        tav_response (json string): The search results in json format
    """

    # For basic search:
    tav_response = tavily_client.search(query=query, max_results=num_results)

    return tav_response



# Example

query = "How much does a bulldog weigh?"

results = run_tavily_search(query, num_results=2)

# Use this str output in the system message example below
# Use this instead of the Eisenhower example
print(results)

{'query': 'How much does a bulldog weigh?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.akc.org/dog-breeds/bulldog/', 'title': 'Bulldog Dog Breed Information - American Kennel Club', 'content': "Bulldogs can weigh up to 50 pounds, but that won't stop them from curling up in your lap, or at least trying to.", 'score': 0.87782305, 'raw_content': None}, {'url': 'https://www.reddit.com/r/Bulldogs/comments/1adz03m/what_does_your_bulldog_weigh/', 'title': 'What does your Bulldog weigh? - Reddit', 'content': "Average weight I believe is 55 and 50 for males and females but they naturally vary. That's just for regular English bulldogs.", 'score': 0.81524754, 'raw_content': None}], 'response_time': 0.78, 'request_id': '57e50c7b-4ab3-494d-a619-3ea8ac2b43dc'}


# Set up the Agents

In [21]:
def run_email_cat_agent(email):

    print("---EMAIL CAT AGENT---")

    system_message = """
    You are a Email Categorizer Agent, You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way.

    You will be provided with an EMAIL. Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
    price_equiry - used when someone is asking for information about pricing \
    customer_complaint - used when someone is complaining about something \
    product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
    customer_feedback - used when someone is giving feedback about a product \
    off_topic when it doesnt relate to any other category \

    Output a single category only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic').
    Format your output as a JSON string with a key named "category".
    eg:
    {
    "category": "price_enquiry"
    }
    """

    # Create the "user" role content
    input_text = f"""
    EMAIL: {email}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input_text)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    print("Email category:", response)

    return response


# Example

email1 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

email2 = """Good day,
I would like to make a booking for a haircut.
Thanks,
George
"""

email3 = """Good day,
Do you still have rooms available on Christmas day?
Thanks,
George
"""

email_cat = run_email_cat_agent(email1)

#print(email_cat)

---EMAIL CAT AGENT---
Email category: customer_feedback


In [23]:
def run_router_agent(email, email_cat):

    print("---ROUTER AGENT---")

    system_message = """
    You are an expert at reading the initial email and routing web search or directly to a draft email. \n
    You will be provided with a CUSTOMER_EMAIL and an EMAIL_CATEGORY.

    Use the following criteria to decide how to route the email: \n\n

    If the initial email only requires a simple response choose 'to_email_writer_agent'.
    If the email is just saying thank you etc then choose 'to_email_writer_agent'.
    Otherwise, use to_research_agent.

    Give a binary choice 'to_research_agent' or 'to_email_writer_agent'. Return the a JSON with a single key 'router_decision' and
    no premable or explaination. Use both the initial email and the email category to make your decision.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    print("Router decision:", response)

    return response



# Example

email1 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

email2 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did. 

By the way, when is the peak tourist season at your resort?

Thanks,
Paul
"""

email = email2

email_cat = run_email_cat_agent(email)

router_decision = run_router_agent(email, email_cat)

#router_decision

---EMAIL CAT AGENT---
Email category: product_enquiry
---ROUTER AGENT---
Router decision: to_research_agent


In [16]:
def run_research_agent(email, email_cat):

    print("---RESEARCH AGENT---")

    system_message = """
    You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    given the CUSTOMER_EMAIL and EMAIL_CATEGORY. Work out the best keywords that will find the best
    info for helping to write a response to the customer email.

    Return a JSON with a single key 'keywords' with no more than 5 keywords and no premable or explaination.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    """

    message_history = create_message_history(system_message, input)

    response = make_llm_api_call(message_history)

    response = json.loads(response)

    keywords_list = (response['keywords'])

    # Run searches using the keywords

    print("Search keywords:\n", keywords_list)
    print("Running tavily search...")

    research_info_list = []

    for query in keywords_list:

        results = run_tavily_search(query, num_results=2)
        research_info_list.append(results)

    print("Research complete.")

    return research_info_list




# Example

email = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email_cat = run_email_cat_agent(email)

research_list = run_research_agent(email, email_cat)

#research_list[0]

---EMAIL CAT AGENT---
Email category: customer_complaint
---RESEARCH AGENT---
Search keywords:
 ['Arizona weather April', 'cloudy overcast weather', 'resort weather Arizona April', 'Arizona April weather average', 'sunshine in Arizona April']
Running tavily search...
Research complete.


In [17]:
def run_email_writer_agent(email, email_cat, research_info_list):

    print("---EMAIL WRITER AGENT---")

    system_message = """
    You are the Email Writer Agent take the CUSTOMER_EMAIL below  from a human that has emailed our company email address, the email_category \
    that the categorizer agent gave it and the research from the research agent and \
    write a helpful email in a thoughtful and friendly way.

    If the customer email is 'off_topic' then ask them questions to get more information.
    If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
    If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
    If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
    If the customer email is 'price_equiry' then try to give the pricing info they requested.

    You never make up information that hasn't been provided by the research_info or in the initial_email.
    Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

    Return a JSON string with a single key 'email_draft' and no premable or explaination.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    RESEARCH_INFO: {research_info_list}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    # Remove leading and trailing spaces
    response = response.strip()

    print("Email writing complete.")
    print("Final email:\n\n", response)

    return response



# Example


email = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email_cat = run_email_cat_agent(email)

research_list = run_research_agent(email, email_cat)

final_email = run_email_writer_agent(email, email_cat, research_list)

#print(final_email)


---EMAIL CAT AGENT---
Email category: customer_complaint
---RESEARCH AGENT---
Search keywords:
 ['Arizona weather April', 'cloudy weather Arizona', 'overcast conditions Arizona', 'April climate Arizona', 'Arizona weather forecast April']
Running tavily search...
Research complete.
---EMAIL WRITER AGENT---
Email writing complete.
Final email:

 Hi George,\n\nThank you for reaching out and sharing your feedback. I'm sorry to hear that the resort weather was overcast and cloudy during your visit – we value your perspective and are committed to ensuring your experiences are as enjoyable as possible.\n\nYour song title, 'Here comes the sun but it never came,' is a brilliant and poetic way to capture the feeling of waiting for clear skies! We'd love to hear more about your creative process if you're ever interested in sharing it.\n\nRegarding your question about Arizona's weather in April: the average high temperatures are around 87°F (31°C) with lows near 55°F (13°C). April is typically a w

# Run the system

In [28]:
email1 = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email2 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""


# Initialize variables
email = email1
research_list = ""

email_cat = run_email_cat_agent(email)

router_decision = run_router_agent(email, email_cat)

if router_decision == "to_research_agent":
    research_list = run_research_agent(email, email_cat)

final_email = run_email_writer_agent(email, email_cat, research_list)

---EMAIL CAT AGENT---
<think>
Okay, let's break down George's email. He starts by saying the resort weather was too cloudy and overcast, which sounds like a complaint. Then he mentions wanting to write a song about the sun not coming, which is a creative expression. Next, he asks about the typical weather in Arizona in April. That's a product enquiry because he's seeking information about a product feature (the resort's weather conditions). Finally, he hopes they fix the issue next time, which is feedback. But the main point here is the question about Arizona's weather. Since the email includes both a complaint and an enquiry, but the primary request is the weather information, which relates to product features (resort's weather), it should be categorized as product_enquiry. However, the question about Arizona's weather might be off-topic if it's not related to the resort's services. Wait, but he's asking about the weather in Arizona in April, which could be relevant if the resort is i